In [21]:
import numpy as np
import math
import random
from tqdm import tqdm
import itertools


In [30]:
def interpolate_lut_trilinear(color, lut, LUT_SIZE):
    # Get color components
    r = color[0] / (2**(8-LUT_SIZE))
    g = color[1] / (2**(8-LUT_SIZE))
    b = color[2] / (2**(8-LUT_SIZE))

    # Convert the inputs to integer indices
    r1 = int(r)
    r2 = min(r1 + 1, lut.shape[0]-1)
    g1 = int(g)
    g2 = min(g1 + 1, lut.shape[1]-1)
    b1 = int(b)
    b2 = min(b1 + 1, lut.shape[2]-1)

    # Get the 8 corner values from the LUT
    f1 = lut[r1, g1, b1]
    f2 = lut[r2, g1, b1]
    f3 = lut[r1, g2, b1]
    f4 = lut[r2, g2, b1]
    f5 = lut[r1, g1, b2]
    f6 = lut[r2, g1, b2]
    f7 = lut[r1, g2, b2]
    f8 = lut[r2, g2, b2]

    # Perform trilinear interpolation for each color component
    interpolated_color = np.empty(3)
    for i in range(3):
        # Change the order of the color channels to BGR
        interpolated_color[i] = f1[i] + \
                                            (r - r1) * (f2[i] - f1[i]) + \
                                            (g - g1) * (f3[i] - f1[i]) + \
                                            (b - b1) * (f5[i] - f1[i]) + \
                                            (r - r1) * (g - g1) * (f4[i] - f1[i]) + \
                                            (r - r1) * (b - b1) * (f6[i] - f1[i]) + \
                                            (g - g1) * (b - b1) * (f7[i] - f1[i]) + \
                                            (r - r1) * (g - g1) * (b - b1) * (f8[i] - f1[i])

    # Scale the output color back to the original range
    interpolated_color *= (2**(8-LUT_SIZE))

    return interpolated_color

In [31]:
from scipy.interpolate import RegularGridInterpolator

def reference_interpolation_function(color, lut, LUT_SIZE):
    # Create a grid of coordinates in the LUT
    grid_coords = [np.arange(2**LUT_SIZE) for _ in range(3)]

    # Create the interpolator function
    interpolator = RegularGridInterpolator(grid_coords, lut)

    # Normalize color
    color = np.array(color) / (2**(8-LUT_SIZE))

    # Interpolate
    interpolated_color = interpolator(color)

    # Scale the output color back to the original range
    interpolated_color *= (2**(8-LUT_SIZE))

    return interpolated_color

In [32]:
def test_lut_interpolation(lut, LUT_SIZE):
    # Initialize the total error and the number of tests
    total_error = 0
    num_tests = 0

    # Generate random colors
    random_colors = [np.random.randint(0, LUT_SIZE, size=3) for _ in range(100)]

    # Generate boundary colors
    boundary_colors = [np.array(color) for color in itertools.product([0, LUT_SIZE-1], repeat=3)]

    # Test the function with the random colors
    for color in tqdm(random_colors, desc="Testing random colors"):
        interpolated_color = interpolate_lut_trilinear(color, lut, LUT_SIZE)
        expected_color = reference_interpolation_function(color, lut, LUT_SIZE)
        error = np.abs(interpolated_color - expected_color).sum() / (expected_color.sum() + 1e-10)  # Add a small constant to avoid division by zero
        total_error += error
        num_tests += 1

        # Check if the relative error is greater than 0.95
        if error > 0.95:
            print(f"Expected color: {expected_color}, Obtained color: {interpolated_color}")

    # Test the function with boundary colors
    for color in tqdm(boundary_colors, desc="Testing boundary colors"):
        interpolated_color = interpolate_lut_trilinear(color, lut, LUT_SIZE)
        expected_color = reference_interpolation_function(color, lut, LUT_SIZE)
        error = np.abs(interpolated_color - expected_color).sum() / (expected_color.sum() + 1e-10)  # Add a small constant to avoid division by zero
        total_error += error
        num_tests += 1

        # Check if the relative error is greater than 0.95
        if error > 0.95:
            print(f"Expected color: {expected_color}, Obtained color: {interpolated_color}")

    # Calculate and print the average relative error
    average_error = total_error / num_tests
    print(f"Average relative error: {average_error}")

In [33]:
# Generate a set of random colors
random_colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for _ in range(1000)]

# Define boundary colors
boundary_colors = [(0, 0, 0), (255, 255, 255), (0, 255, 0), (255, 0, 255), (255, 255, 0)]

# Test with LUTs of different sizes
for LUT_SIZE in range(6, 7):
    # Create a test LUT
    lut = np.zeros((2**LUT_SIZE, 2**LUT_SIZE, 2**LUT_SIZE, 3), dtype=np.uint8)
    for i in range(2**LUT_SIZE):
        for j in range(2**LUT_SIZE):
            for k in range(2**LUT_SIZE):
                lut[i, j, k, 0] = i * (256 // 2**LUT_SIZE)
                lut[i, j, k, 1] = j * (256 // 2**LUT_SIZE)
                lut[i, j, k, 2] = k * (256 // 2**LUT_SIZE)

    print(f"\nTesting with LUT size: {LUT_SIZE}")
    print(lut.shape)
    test_lut_interpolation(lut, LUT_SIZE)


# List of LUT file paths to test
path_root = "C:/Users/thoma/Documents/MODAL/LUTCreator/"
lut_file_paths = ["ReinhardLUTs/inverse_reinhard_lut_int_128.txt",
                  "ReinhardLUTs/reinhard_lut_int_128.txt"]
LUT_SIZE_IMPORT = 7  # Replace with your actual LUT size

# Test with imported LUTs
for lut_file_path in lut_file_paths:
    # Load the existing LUT
    lut = np.loadtxt(path_root + lut_file_path, dtype=int)
    lut = lut.reshape((2**LUT_SIZE_IMPORT, 2**LUT_SIZE_IMPORT, 2**LUT_SIZE_IMPORT, 3))

    # Permute the color channels from BGR to RGB
    lut = lut[..., [1, 0, 2]]

    print(f"\nTesting with imported LUT from file: {lut_file_path}")
    print(lut.shape)
    test_lut_interpolation(lut, LUT_SIZE_IMPORT)


Testing with LUT size: 6
(64, 64, 64, 3)


Testing random colors:  44%|████▍     | 44/100 [00:00<00:00, 434.96it/s]

Expected color: [[ 8.  4. 12.]], Obtained color: [17.5 10.5 22.5]
Expected color: [[ 4.  8. 12.]], Obtained color: [10.5 17.5 22.5]
Expected color: [[ 4. 12. 12.]], Obtained color: [12.25 26.25 26.25]
Expected color: [[12.  4.  8.]], Obtained color: [22.5 10.5 17.5]
Expected color: [[12.  8.  4.]], Obtained color: [22.5 17.5 10.5]
Expected color: [[ 4. 12. 12.]], Obtained color: [12.25 26.25 26.25]


Testing random colors:  92%|█████████▏| 92/100 [00:00<00:00, 460.94it/s]

Expected color: [[12. 12.  8.]], Obtained color: [31.5 31.5 24.5]

...

Testing boundary colors: 100%|██████████| 8/8 [00:00<00:00, 77.24it/s]


Average relative error: 0.036179377996809314

Testing with imported LUT from file: ReinhardLUTs/reinhard_lut_int_128.txt
(128, 128, 128, 3)


Testing boundary colors: 100%|██████████| 8/8 [00:00<00:00, 71.14it/s]

Average relative error: 0.07784241221661183


: 